In [1]:
import config

In [2]:
import spotipy
import json
import pprint
from bs4 import BeautifulSoup
import requests
import numpy as np
import pprint
from pyjarowinkler import distance
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.c_id,
                                                           client_secret= config.c_se))
import pandas as pd
import pickle
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from IPython.display import IFrame

In [3]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!") 


In [4]:
# loading top charts
top_charts = pd.read_csv('Top Charts.csv')

In [5]:
# loading scaler
scaler = load("Model/scaler.pickle")

In [6]:
# loading kmeans
kmeans = load("Model/kmeans_22.pickle")

In [7]:
# loading full dataframe
all_info_df = pd.read_csv('all_info_df.csv')
all_info_df.head()

,title,artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,cluster
0,Crazy - Single Version,"Patsy Cline, The Jordanaires",3zpj9dvJABiyMrmLCPw6i8,0.471,0.021,10,-24.385,1,0.0366,0.931,0.000001,0.1030,0.356,107.796,3
1,Folsom Prison Blues,"Johnny Cash, The Tennessee Two",121sjkub4p4zcFNozkKnQD,0.754,0.540,5,-8.326,1,0.0266,0.687,0.000075,0.0868,0.790,101.738,10
2,I Will Always Love You,Dolly Parton,6nRvgo4ySezQKD1D7NIqAu,0.446,0.142,9,-13.498,1,0.0335,0.960,0.001140,0.1010,0.156,135.724,3
3,Springsteen,Eric Church,05ZACkzW8YbRq3eFgIfSNB,0.692,0.809,2,-5.935,1,0.0255,0.100,0.000098,0.1850,0.927,104.023,15
4,He Stopped Loving Her Today,George Jones,6qMgx8ys4MNNxypaIvbG0i,0.430,0.173,8,-18.597,1,0.0321,0.594,0.000111,0.2240,0.365,72.103,3


In [19]:
# song recommendation function
def song_rec():
    user_input = input("Please input a song: ")
    print('\n')
    song_list_cleaned = [each_string.replace(" ","").lower() for each_string in list(top_charts.title)]
    artist_list_cleaned = [each_string.replace(" ","").lower() for each_string in list(top_charts.artist)]
    user_input_cleaned = user_input.replace(" ", "").lower()
    similarity_count = 0
    duplicate_check = top_charts[top_charts.title.duplicated(keep=False)]
    duplicate_list = [each_string.replace(" ","").lower() for each_string in list(duplicate_check.title)]
    similarity_song_list = []
    similarity_score_list = []
    for i in range(0, len(top_charts.title)):
        similarity_score = distance.get_jaro_distance(user_input_cleaned, song_list_cleaned[i], winkler=True, scaling=0.1)
        similarity_song_list.append(top_charts.title[i])
        similarity_score_list.append(similarity_score)
        if similarity_score > 0.9:
            similarity_count = similarity_count+1
        else:
            similarity_count = similarity_count
    similarity_df = pd.DataFrame({"title":similarity_song_list, "similarity":similarity_score_list})
    user_input_corrected = similarity_df.loc[similarity_df.similarity == max(similarity_df.similarity), "title"].values[0]
    user_input_corrected_and_cleaned = user_input_corrected.replace(" ","").lower()
    if similarity_count>0:
        suggestion = top_charts[top_charts["title"]!=user_input_corrected].sample(1)
        song_suggestion = suggestion.iloc[0][0]
        by_artist = suggestion.iloc[0][1]
        song_suggestion_id = sp.search(q=song_suggestion ,limit=1,market="GB")['tracks']['items'][0]['id']
        if user_input_corrected_and_cleaned in duplicate_list:
            print(duplicate_check)
            artist_user_input = input("Please input the artist: ")
            artist_user_input_cleaned = artist_user_input.replace(" ", "").lower()
            suggestion2 = top_charts[(top_charts["title"]!=user_input_corrected) | (top_charts["artist"]!=artist_user_input)].sample(1)
            song_suggestion2 = suggestion2.iloc[0][0]
            by_artist2 = suggestion2.iloc[0][1]
            if artist_user_input_cleaned in artist_list_cleaned:
                return (song_suggestion2 + " - " + by_artist2)
            else:
                return "Please input a valid artist"
        else:
            print("Your song is in the top charts! Here is another one for you.")
            return IFrame(src="https://open.spotify.com/embed/track/"+song_suggestion_id, width="320", height="80", frameborder="0", allowtransparency="true", allow="encrypted-media",)
    else:
        try:
            result = sp.search(q=user_input ,limit=5,market="GB")
            for i in range(0,len(result['tracks']['items'])):
                print(result['tracks']['items'][i]['artists'][0]['name']+" ("+result['tracks']['items'][i]['name']+")")
            artist_input = input("Which artist do you mean? Please copy the artist name from above: ")
            new_result = sp.search(q=(user_input+" "+artist_input) ,limit=1,market="GB")
            new_result_uri = new_result['tracks']['items'][0]['uri']
            my_dict = sp.audio_features(new_result_uri)[0]
            my_dict_new = { key: [my_dict[key]] for key in list(my_dict.keys()) }
            user_input_af_df = pd.DataFrame(my_dict_new)
            user_input_af_df.drop(columns=['type','uri','track_href','analysis_url','duration_ms','time_signature','id'],inplace=True)
            user_input_af_df_tf = scaler.transform(user_input_af_df)
            user_input_cluster = kmeans.predict(user_input_af_df_tf)[0]
            track_id = all_info_df[all_info_df['cluster']==user_input_cluster].sample(1)['id'].values[0]
            print('\n')
            print('Here is another similar song for you.')
            return IFrame(src="https://open.spotify.com/embed/track/"+track_id, width="320", height="80", frameborder="0", allowtransparency="true", allow="encrypted-media",)
        except:
            print("Song not found!") 

In [21]:
# user input
song_rec()

Please input a song: sdfkljjks


Which artist do you mean? Please copy the artist name from above: sdfsdf
Song not found!
